# LSTM Model to Predict Sales
In this notebook, we'll be using a LSTM model to attempt to predict sales. 

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from sales_data import SalesData
from lstm import ModelInput, LSTMModel
import matplotlib.pyplot as plt

## Fetch Sales, Shop, and Item data
The class SalesData fetches all data regarding shops, items, and sales. It also imports the ```nlp``` module to break out what is assumed to be the locations and types of the shops using traditional NLP techniques. Ultimately, we end up with categorical info on both the shops and the items for further exploration of the data. Additionally, sales data is available at both daily and monthly granularities.

In [ ]:
sd = SalesData()

sd.set_sales_data()
sd.merge_shop_data_to_sales()
sd.merge_item_data_to_sales()

In [ ]:
sd.monthly_sales.head()

## Build Input for LSTM Model
Class ModelInput ingests ```sd.monthly_sales``` and breaks up the data according to shop groups determined by their total sales with the intent of modeling each group separately.

In [ ]:
mi = ModelInput(sd.monthly_sales)

# mi.set_shop_groups()
# mi.date_to_cols(mi.shop_grp_1)
mi.date_to_cols(mi.data)
mi.stack_periods()
mi.train_test_split()
mi.scale_min_max()
mi.reshape()

## Train LSTM Model

In [ ]:
model = LSTMModel(mi.X_train, mi.y_train)

In [ ]:
model.add_layers()
model.fit(epochs=50)

In [ ]:
y_pred = model.predict(mi.X_test)

In [ ]:
mi.test_results(y_pred)  # for entire on 30 epochs, 2 lstm layers, 50 units

# Test Set
Merge test set to 2015 data. Then get predictions.

In [ ]:
test_set = pd.read_csv('../data/test.csv')

test_set['shop_item'] = \
    test_set['shop_id'].astype(str) + '_' + test_set['item_id'].astype(str)

test_df = mi.current_year_df.reset_index()

In [ ]:
test_df.head()

In [ ]:
test_input = pd.merge(test_set.loc[:, 'shop_item'], 
                    test_df, 
                    on='shop_item', how='left')

test_input.fillna(0, inplace=True)

In [ ]:
test_input.head()

In [ ]:
X_test = test_input.iloc[:, 1:]
X_test = mi.X_scaler.transform(X_test)
# X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
y_pred = model.predict(X_test)
y_pred = mi.y_scaler.inverse_transform(y_pred)

In [ ]:
test_set['item_cnt_month'] = y_pred
test_set.head()

In [ ]:
output = test_set.loc[:, ['ID', 'item_cnt_month']].copy()
output.set_index(keys='ID', inplace=True)
output = output.clip(0, 20)

output.to_csv('../output/lstm_output_v2.csv', index=True)